<a href="https://colab.research.google.com/github/MomPansy/ID2223FinalProj/blob/main/ID2209FinalProj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [21]:
from openai import OpenAI
import random
import json
client = OpenAI()

tools = [
    {
        'type': 'function',
        'function': {
          'name': 'compare_ask_bid',
          'description': 'checks if the ask price is lower than the bid price',
          'parameters': {
              'type': 'object',
              'properties': {
                  'ask_price': {
                      'type': 'string',
                      'description': 'current ask price of the auctioned item'
                    },
                  'bid_price': {
                      'type': 'string',
                      'description': 'bid price of current guest'
                  }
              },
              'required': ['ask_price', 'bid_price']
          }
        }
    }
]

def compare_ask_bid(ask_price, bid_price):
  # Remove non-numeric characters (like $) from the strings
  ask_price = ''.join(filter(str.isdigit, ask_price))
  bid_price = ''.join(filter(str.isdigit, bid_price))

  # Convert to integers
  ask_price, bid_price = int(ask_price), int(bid_price)

  # Compare and return the result
  if ask_price < bid_price:
      return True, ask_price
  else:
      return False, ask_price

class AuctioneerAgent:
  def __init__(self, name, start_price = 5000):
    self.name = name
    self.start_price = start_price
    self.conversation_history = []
    self.system_prompt =f"""
      You are a dutch auctioneer with the name {self.name}.

      You will output messages of the form
      Text: '''
        {self.name}:
        Message:
        Ask Price:
      '''

      You will conduct the auction with a starting ask price of ${self.start_price}.
      If for all participants, they are NOT placing a bid, you will always lower your ask price by 500.

      Else, reply with
      Text: '''
        {self.name}: SOLD to <highest bidder> at <bid_price>
        TASK_COMPLETE
      '''
      """
    self.step('system', self.system_prompt)

  def add_to_conversation(self, message):
    self.conversation_history.append(message)

  def step(self, role, content):
    "role = user|assistant|system"
    message = {
        'role': role,
        'content': content
    }
    self.add_to_conversation(message)

    response = client.chat.completions.create(
      model="gpt-4",
      messages=self.conversation_history,
    )
    return response.choices[0].message.content

  def reset(self):
    self.conversation_history = []

class GuestAgent:
  def __init__(self, name):
    self.name = name
    self.bid_price = random.randint(2000,4000)
    self.conversation_history = []
    self.system_prompt =f"""
      You are a partcipant of an auction with the name {self.name}.

      You will output messages of the form
      Text: '''
        {self.name}:
        Place a bid: (YES / NO)
      '''
      If the ask price is lower than ${self.bid_price}, set Place a bid: YES.
      Else set Place a bid: NO.

      """
    self.add_to_conversation({
        'role': 'system',
        'content':self.system_prompt
    })

  def add_to_conversation(self, message):
    self.conversation_history.append(message)

  def step(self, role, content):
    "role = user|assistant|system"
    prompt = f'{self.name}: ${self.bid_price}'
    content += '\n' + prompt
    message = {
        'role': role,
        'content': content
    }
    self.add_to_conversation(message)

    response = client.chat.completions.create(
      model="gpt-4",
      messages=self.conversation_history,
      tools = tools,
      tool_choice = {'type':'function', 'function':{'name': 'compare_ask_bid'}}
    )

    place_bid, ask_price = self.format_response(response)
    self.ask_price = ask_price
    if place_bid == False:
      return f'{self.name}: Not placing a bid'
    else:
      return f'{self.name}: Placing a bid of ${self.bid_price}'

  def reset(self):
    self.conversation_history = []

  def format_response(self, response):
    response = response.json()
    response = json.loads(response)
    chosen_function = eval(response['choices'][0]['message']['tool_calls'][0]['function']['name'])
    kwargs = json.loads(response['choices'][0]['message']['tool_calls'][0]['function']['arguments'])
    reply, ask_price = chosen_function(**kwargs)
    return reply, ask_price

class Simulation:
  def __init__(self, guests):
    self.guests = guests
    self.msglog = ''

  def stepall(self, auctioneerMsg):
    self.reset()
    for guest in self.guests:
      self.msglog += guest.step('user', auctioneerMsg) + '\n'
      self.ask_price = guest.ask_price
    self.msglog = f'Current turn ask price: {self.ask_price}' + '\n' + self.msglog
    self.msglog += 'END TURN'
    return self.msglog

  def reset(self):
    self.msglog = ''



In [22]:
Auctioneer = AuctioneerAgent(name='John')
auctioneer_response = Auctioneer.step('system', Auctioneer.system_prompt)
print(f"Auctioneer John's Initialization: \n{auctioneer_response}\n")

guests = [GuestAgent(name='Terry'), GuestAgent(name='Nicole'), GuestAgent(name='Brad')]
manager = Simulation(guests)

for i in range(15):
    print(f"Round {i+1}: Guest Response")
    guest_response = manager.stepall(auctioneer_response)
    print(guest_response + "\n")

    print(f"Round {i+1}: Auctioneer John's Response")
    auctioneer_response = Auctioneer.step('user', guest_response)
    print(auctioneer_response + "\n")
    print('-'*40)

    if 'TASK_COMPLETE' in auctioneer_response:
        print('Task has been completed')
        break


Auctioneer John's Initialization: 
Text: '''
John:
Message: The auction will start at $5000. Who wants to place a bid?
Ask Price: $5000
'''

Round 1: Guest Response
Current turn ask price: 5000
Terry: Not placing a bid
Nicole: Not placing a bid
Brad: Not placing a bid
END TURN

Round 1: Auctioneer John's Response
Text: '''
  John:
  Message: Noted. No bids placed at this price. The bid price will be decreased by $500.
  Ask Price: $4500
'''

----------------------------------------
Round 2: Guest Response
Current turn ask price: 4500
Terry: Not placing a bid
Nicole: Not placing a bid
Brad: Not placing a bid
END TURN

Round 2: Auctioneer John's Response
Text: '''
John:
Message: No one placed a bid in this round. The ask price will be reduced further.
Ask Price: $4000
'''

----------------------------------------
Round 3: Guest Response
Current turn ask price: 4000
Terry: Not placing a bid
Nicole: Not placing a bid
Brad: Not placing a bid
END TURN

Round 3: Auctioneer John's Response
Tex